In [1]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import glob
import os
import hera_pspec as hp
from pyuvdata import UVBeam, UVData, UVCal, UVFlag, utils as uvutils
import copy
import sys
from astropy import constants, units, cosmology
import uvtools as uvt
from collections import OrderedDict as odict
from hera_pspec.data import DATA_PATH

In [2]:
from pspec_likelihood import *

In [3]:
%load_ext autoreload
%autoreload 2

## 1. Form UVPSpec objects to fit

Import test data

In [4]:
path_to_wf = '../tests/data/'
dfile = 'data_calibrated_testfile.h5'

In [5]:
uvd = UVData()
uvd.read_uvh5(os.path.join(path_to_wf, dfile))

# # beam 
beamfile = os.path.join(DATA_PATH, 'HERA_NF_pstokes_power.beamfits')
uvb = hp.pspecbeam.PSpecBeamUV(beamfile)
# Create a new PSpecData object, and don't forget to feed the beam object
ds = hp.PSpecData(dsets=[uvd, uvd], wgts=[None, None], beam=uvb)
ds.Jy_to_mK()
# choose baselines
baselines1, baselines2, blpairs = hp.utils.construct_blpairs(uvd.get_antpairs(),
                                                            exclude_permutations=True,
                                                            exclude_auto_bls=True)

Cannot convert dset 1 Jy -> mK because vis_units = mK


Form power spectrum

In [6]:
# compute ps
uvp = ds.pspec(baselines1, baselines2, dsets=(0, 1), pols=[('pI', 'pI')], 
               spw_ranges=(175, 195), taper='bh', verbose=False,
               store_cov=True, store_window=True, baseline_tol=100.)
print(uvp.Nblpairs, uvp.data_array[0].shape, uvp.Ntimes)

Producing time-uniform covariance matrices between bandpowers.
Casting complex values to real discards the imaginary part


15 (30, 20, 1) 2


In [8]:
# get redundant groups
blpair_groups, blpair_lens, _ = uvp.get_red_blpairs()

In [9]:
# there are baseline pairs that do not belong to a redundant group
extra_blpairs = set(uvp.blpair_array) - set([blp for blpg in blpair_groups for blp in blpg])
print(len(extra_blpairs))

9


In [10]:
# only keep blpairs in redundant groups
uvp.select(blpairs=[blp for blpg in blpair_groups for blp in blpg])
print(uvp.Nblpairs, uvp.data_array[0].shape)

6 (12, 20, 1)


In [11]:
# perform redundant average
uvp.average_spectra(blpair_groups=blpair_groups, time_avg=True)
print(uvp.Nblpairs, uvp.data_array[0].shape)

2 (2, 20, 1)


Take spherical average

In [12]:
kbins = np.linspace(0.1, 2.5, 40)
sph = hp.grouping.spherical_average(uvp, kbins, np.diff(kbins).mean())

## 2. Generate likelihood

In [13]:
def theory_model(z, k, params):
    return k**3/(2*np.pi**2)*params[0] * un.mK**2 *(1.+z)/k **params[1]

In [14]:
def bias(k, z, little_h=True):
    return np.ones(k.shape)

In [15]:
def bias_prior(params):
    return 1.

In [17]:
dmi = DataModelInterface.from_uvpspec(uvp=uvp,
                                      spw=0,
                                      theory_model=theory_model,
                                      theory_uses_spherical_k=True)

Converting to Delta^2 in place...
